In [ ]:
!pip install -q sentence-transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
#Data load
train_df = pd.read_csv('HF_new_traindata.csv')
test_df = pd.read_csv('HF_new_testdata.csv')

In [6]:
assert 'question' in train_df.columns and 'answer' in train_df.columns, "Train file must have 'question' and 'answer'"
assert 'question' in test_df.columns and 'answer' in test_df.columns, "Test file must have 'question' and 'answer'"

# Merge datasets
full_df = pd.concat([train_df, test_df], ignore_index=True)
questions = full_df['question'].tolist()
answers = full_df['answer'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load sentence-transformer BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # small and fast, good for semantic search

# Create embeddings
question_embeddings = model.encode(questions, convert_to_tensor=False)

# Create FAISS index
dimension = question_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))

print("✅ Model loaded, embeddings created from train+test data")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded, embeddings created from train+test data


In [8]:
def get_answer(user_question, top_k=1):
    query_embedding = model.encode([user_question], convert_to_tensor=False)
    distances, indices = index.search(np.array(query_embedding), top_k)
    idx = indices[0][0]
    return answers[idx], questions[idx]

In [12]:
while True:
    user_input = input("🩺 Ask a healthcare question (or type 'exit'): ")
    if user_input.lower() == 'exit':
        print("👋 Chat ended.")
        break
    answer, matched_q = get_answer(user_input)
    print(f"\n🤖 Matched Question: {matched_q}")
    print(f"✅ Answer: {answer}\n")

🩺 Ask a healthcare question (or type 'exit'): what is parcitamole use

🤖 Matched Question: The drug cilostazol is known for its ability to relax vascular smooth muscle and therefore cause vasodilation through its inhibition of phosphodiesterase 3. Given this mechanism of action, what other effect would be expected?
✅ Answer: Positive inotropy

🩺 Ask a healthcare question (or type 'exit'): exit
👋 Chat ended.
